# Projet Bigdata 2021-2022: GDELT: Les evenements marquants de l’annee 2021

Pierre Dal Bianco \
Aurélien Kouadio \
Jia Liao \
Pooran Shahdi \
Philéas Samir \
Gwladys Sanchez

In [1]:
import pandas as pd
from cassandra.cluster import Cluster

In [2]:
cluster = Cluster(['tp-hadoop-1'])
session = cluster.connect('test')

**Question a.** afficher le nombre d’articles/évènements qu’il y a eu pour chaque triplet (jour, pays de l’évènement, langue de l’article).

In [10]:
query_a = session.execute("SELECT * FROM table_ab;")
data_a = pd.DataFrame(query_a)

In [11]:
data_a.head()

,pays,annee,mois,jour,event_id,langue,total
0,JE,2021,202111,20211127,1020807020,eng,1
1,JE,2021,202111,20211129,1021093574,eng,1
2,JE,2021,202112,20211221,1020962681,eng,4
3,JE,2021,202112,20211225,1020682973,spa,1
4,JE,2021,202112,20211226,1020782079,spa,1


**Question b.** pour un pays donné en paramètre, affichez les évènements qui y ont eu place triées par le nombre de mentions (tri décroissant); permettez une agrégation par jour/mois/année

In [16]:
pays = 'FR'
agregation = 'annee' # 'annee, mois' ou 'annee, mois, jour'

query_b = session.execute(
    f"""SELECT event_id, SUM(total) as compte, {agregation}
    FROM table_ab
    WHERE pays = '{pays}'
    GROUP BY {agregation};""")
data_b = pd.DataFrame(query_b).sort_values('compte')

In [17]:
data_b.head()

,event_id,compte,annee
0,1020679947,18,2021


**Question c.** pour une source de donnés passée en paramètre (gkg.SourceCommonName) affichez les thèmes, personnes, lieux dont les articles de cette sources parlent ainsi que le nombre d’articles et le ton moyen des articles (pour chaque thème/personne/lieu); permettez une agrégation par jour/mois/année.

In [24]:
source = 'lemonde.fr'
agregation = 'annee' # 'annee, mois' ou 'annee, mois, jour'

query_c = session.execute(
    f"""SELECT source, theme, personne, lieu, SUM(total) AS somme_total, SUM(somme_ton)  AS somme_ton, {agregation}
    FROM table_c
    WHERE source = '{source}'
    GROUP BY theme, personne, lieu, {agregation};""")
data_c = pd.DataFrame(query_c)
data_c['moyenne_ton'] = data_c['somme_ton'] / data_c['somme_total'] 

In [25]:
data_c.head()

,source,theme,personne,lieu,somme_total,somme_ton,annee,moyenne_ton
0,lemonde.fr,AFFECT,UNK,BM,1,-2.884615,2021,-2.884615
1,lemonde.fr,AFFECT,UNK,FR,1,-3.099511,2021,-3.099511
2,lemonde.fr,AFFECT,UNK,LE,1,-3.783784,2021,-3.783784
3,lemonde.fr,AFFECT,UNK,MY,1,-2.884615,2021,-2.884615
4,lemonde.fr,AFFECT,UNK,RP,1,-3.099511,2021,-3.099511


**Question d.** étudiez l’évolution des relations entre deux pays (specifies en paramètre) au cours de l’année. Vous pouvez vous baser sur la langue de l’article, le ton moyen des articles, les themes plus souvent citées, les personalités ou tout element qui vous semble pertinent.

In [26]:
langue = 'eng'
lieu = 'FR'
agregation = 'annee' # 'annee, mois' ou 'annee, mois, jour'

query_d = session.execute(
    f"""SELECT langue, lieu, SUM(total) AS somme_total, SUM(somme_ton) AS somme_ton, {agregation}
    FROM table_d
    WHERE langue = '{langue}' AND lieu = '{lieu}'
    GROUP BY {agregation};""")
data_d = pd.DataFrame(query_d)
data_d['moyenne_ton'] = data_d['somme_ton'] / data_d['somme_total'] 

In [27]:
data_d.head()

,langue,lieu,somme_total,somme_ton,annee,moyenne_ton
0,eng,FR,21913,-19100.371912,2021,-0.871646


In [28]:
# on ferme la session à la fin du fichier
session.shutdown()